# Fake News Analysis

# Importing Required Libraries

In [5]:
#modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import  roc_curve, auc
import string
import re

# Loading the Dataset

In [8]:
fake_data=pd.read_csv("Fake.csv")
true_data=pd.read_csv("True.csv")

In [10]:
#First five columns and last five columns of fake and true dataset
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [11]:
#last five rows of fake_data
fake_data.tail()

,title,text,subject,date
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016"


In [12]:
#first five rows of true data
true_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [13]:
#last five rows of true data
true_data.tail()

,title,text,subject,date
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017"


# Labeling and Combining Data

In [15]:
# Add Label (fake=0,real=1)
fake_data["label"]=0
true_data["label"]=1

In [16]:
#shape of both
fake_data.shape

(23481, 5)

In [17]:
true_data.shape

(21417, 5)

In [19]:
#merge two data file and make one
df=pd.concat([fake_data,true_data], axis=0)

In [20]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [21]:
df.shape

(44898, 5)

In [23]:
df.dtypes

title      object
text       object
subject    object
date       object
label       int64
dtype: object

In [24]:
#columns
df.columns

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

# Text Cleaning & Preprocessing

In [26]:
data=df.drop(["title","subject","date"], axis=1)

In [27]:
#Again shape
data.shape

(44898, 2)

In [29]:
print(df["title"])

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
21412    'Fully committed' NATO backs new U.S. approach...
21413    LexisNexis withdrew two products from Chinese ...
21414    Minsk cultural hub becomes haven from authorities
21415    Vatican upbeat on possibility of Pope Francis ...
21416    Indonesia to buy $1.14 billion worth of Russia...
Name: title, Length: 44898, dtype: object


In [30]:
print(df["label"])

0        0
1        0
2        0
3        0
4        0
        ..
21412    1
21413    1
21414    1
21415    1
21416    1
Name: label, Length: 44898, dtype: int64


In [31]:
#shufle the data
data=data.sample(frac=1).reset_index(drop=True)

In [32]:
data.head()

,text,label
0,"The Berkeley, California anti-Trump protesters...",0
1,"So, if the climate scare-mongers can t fright...",0
2,"By ramping up US troop levels in Afghanistan,...",0
3,J.R. Smith 21st Century WireFor the last two ...,0
4,Peter Certo Other WordsEven in an election yea...,0


In [33]:
data.head(50)

,text,label
0,"The Berkeley, California anti-Trump protesters...",0
1,"So, if the climate scare-mongers can t fright...",0
2,"By ramping up US troop levels in Afghanistan,...",0
3,J.R. Smith 21st Century WireFor the last two ...,0
4,Peter Certo Other WordsEven in an election yea...,0
5,White House counselor Kellyanne Conway has pre...,0
6,Trump wins big in New Hampshire in what is an ...,0
7,Just about every journalist who was waiting to...,0
8,ISLAMABAD (Reuters) - A hardline Pakistani Isl...,1
9,Australian-born Creationist Ken Ham and his fu...,0


In [34]:
#Check null values
data.isnull().sum()

text     0
label    0
dtype: int64

In [38]:
data.duplicated().sum()

np.int64(6251)

In [39]:
data=data.drop_duplicates().reset_index(drop=True)

In [40]:
#again check
data.duplicated().sum()

np.int64(0)

In [42]:
data.shape

(38647, 2)

In [43]:
#text preprocessing
#This function will done to remove all punctuation, commas and also headind,style,https and many mores
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]','', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\s+|www\.\S+', '', text)
    text = re.sub('<.*?>+','', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n','', text)
    text = re.sub('\w*\d\w*','', text)
    return text

In [45]:
#apply on data
data["text"]=data["text"].apply(wordopt)

In [ ]:
data.head(10)

# Splitting Data into Train & Test

In [47]:
x=data["text"]
y=data["label"]

In [48]:
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.25)

In [49]:
# Convert text into numerical ways that model learn out best
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Model Training (Logistic Regression, etc.)

In [50]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)
pred_lr = LR.predict(xv_test)
LR.score(xv_test, y_test)

0.9828192920720348

In [51]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4399
           1       0.98      0.99      0.98      5263

    accuracy                           0.98      9662
   macro avg       0.98      0.98      0.98      9662
weighted avg       0.98      0.98      0.98      9662



In [52]:
# Decision tree
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)

0.9941006002897951

In [53]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4399
           1       0.99      1.00      0.99      5263

    accuracy                           0.99      9662
   macro avg       0.99      0.99      0.99      9662
weighted avg       0.99      0.99      0.99      9662



In [54]:
# Gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)
predit_gb = GB.predict(xv_test)
GB.score(xv_test, y_test)

0.9958600703788035

In [55]:
print(classification_report(y_test, predit_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4399
           1       0.99      1.00      1.00      5263

    accuracy                           1.00      9662
   macro avg       1.00      1.00      1.00      9662
weighted avg       1.00      1.00      1.00      9662



In [56]:
# Random forest
from sklearn.ensemble import RandomForestClassifier
RF =  RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)
pred_rf = RF.predict(xv_test)
RF.score(xv_test, y_test)

0.9835437797557441

In [57]:
def output_lable(n):
    if n ==0:
        return "Fake News"
    elif n==1:
        return "Not A Fake News" 
def testing(news):
    testing_news = {"test":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["test"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)



    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(output_lable(pred_LR[0]), output_lable(pred_DT[0]), output_lable(pred_GB[0]), output_lable(pred_RF[0])))

In [61]:
news=str(input())
testing(news) 

 hello




LR Prediction: Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Fake News
